# Стреляем себе в ногу
 
* Давайте разберём на Keras несколько архитектур. Попытайтесь понять что именно с ними не так. 
* Упражнения позаимствованы [из нескольких ШАДОвских семинаров по Keras.](https://github.com/yandexdataschool/Practical_DL/blob/master/week03_convnets/other_frameworks/how_to_shoot_yourself_in_the_foot_with_cnn.ipynb)

#### a) Задача регрессии. Предсказываем цены на недвижимость.

In [ ]:
model = keras.models.Sequential()
model.add(L.InputLayer([39]))

# лучше нормализовать входы (не ошибка)
model.add(L.BatchNormalization())                         

# полносвязный слой без нелинейности (нейросеть не лучше линейной регрессии)
model.add(L.Dense(128, kernel_initializer=keras.initializers.zeros()))

# так ещё и инициализируется нулями (эффект симметрии, нейроны будут одинаковыми)
model.add(L.Dense(128, kernel_initializer=keras.initializers.zeros()))
model.add(L.Dense(1))

# попробовать relative mse, log-mse (не ошибка)
model.compile(optimizer='sgd', loss='mean_squared_error') 

####  б) Классификация картинок, например, fashion MNIST

In [ ]:
model = keras.models.Sequential()
model.add(L.InputLayer([28, 28, 1]))

# многовато фильтров, вряд ли найдётся 512 значимо разных 3x3 карты
model.add(L.Conv2D(filters=512, kernel_size=(3, 3)))
model.add(L.Activation('relu'))

# relu и max pool можно переставить местами, будет то же самое чуть быстрее (не ошибка)
model.add(L.MaxPool2D(pool_size=(2, 2)))
model.add(L.Flatten())
model.add(L.Dense(100))

# softmax плохо подходит для промежуточной нелинейности (почти все нейроны будут близки к 0)
model.add(L.Activation('softmax'))

model.add(L.Dropout(0.1)) # хороший, годный дропаут
model.add(L.Dense(10))
model.add(L.Activation('softmax'))

# не стоит dropout-ить вероятности на выходе, рискуете получить бесконечный лосс
model.add(L.Dropout(0.1))

# обучать классификацию по квадратичной ошибке можно, но... это безбожно
model.compile(optimizer='rmsprop', loss='mean_squared_error')

#### в) И снова fashion MNIST, но теперь мы знаем, что размер картинки $100 \times 100$ пикселей

Если попробовать скомпилировать эту сетку, вылезет ошибка.

In [ ]:
model = keras.models.Sequential()
model.add(L.InputLayer([100, 100, 3]))

for filters in [32, 64, 128, 256]:
    # на четвёртой итерации вход станет меньше 5x5, свёртка "не влезет"
    model.add(L.Conv2D(filters, kernel_size=(5, 5)))
    model.add(L.Conv2D(filters, kernel_size=(1, 1)))
    
    # свёртка 1x1 имела бы смысл, если бы перед ней была нелинейность
    model.add(L.MaxPooling2D(pool_size=(3, 3)))
    model.add(L.Activation('relu'))
    model.add(L.BatchNormalization())
    # не ошибка, но рекоммендуют вставлять BatchNorm перед активацией (в интернетах много срачей на эту тему)

model.add(L.Flatten())

model.add(L.Dense(100, activation='relu'))
model.add(L.Dropout(0.5))
model.add(L.Dense(10, activation='softmax'))

# accuracy имеет нулевую производную, поэтому оптимизировать лучше кроссэнтропию
# (sparse_categorical_crossentropy)
model.compile(optimizer='adam', loss='sparse_categorical_accuracy')